In [1]:
import pandas as pd
import numpy as np
import joblib
from haversine import haversine, Unit
from datetime import datetime, timedelta

# --- Functions from Phase 5 ---
def calculate_distance(lat1, lon1, lat2, lon2):
    return haversine((lat1, lon1), (lat2, lon2), unit=Unit.KILOMETERS)

def calculate_net_profit(predicted_price_per_kg, quantity_tonnes, distance_km, market_fees_percent=1.5):
    total_revenue = predicted_price_per_kg * quantity_tonnes * 1000
    transport_cost = (distance_km * 2 * quantity_tonnes) + 500
    market_fees = total_revenue * (market_fees_percent / 100)
    other_costs = 200 * quantity_tonnes
    total_costs = transport_cost + market_fees + other_costs
    net_profit = total_revenue - total_costs
    return {
        'revenue': round(total_revenue, 2),
        'total_costs': round(total_costs, 2),
        'net_profit': round(net_profit, 2)
    }

# --- Load our dataset with features to get the latest data for prediction ---
# This simulates having a database of up-to-date market information.
df_features = pd.read_csv('data/market_prices.csv')
df_features['date'] = pd.to_datetime(df_features['date'])
# (We are re-using the feature creation logic from Phase 3 inside the recommender)
def create_features(df):
    df = df.sort_values(['market_name', 'commodity', 'date'])
    df['price_yesterday'] = df.groupby(['market_name', 'commodity'])['modal_price'].shift(1)
    df['price_last_week'] = df.groupby(['market_name', 'commodity'])['modal_price'].shift(7)
    df['price_avg_7days'] = df.groupby(['market_name', 'commodity'])['modal_price'].shift(1).rolling(7).mean()
    df['day_of_week'] = df['date'].dt.dayofweek
    df['month'] = df['date'].dt.month
    df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
    df['arrivals_yesterday'] = df.groupby(['market_name', 'commodity'])['arrivals_qty'].shift(1)
    df = df.dropna()
    return df
df_features = create_features(df_features)


def recommend_markets(farmer_lat, farmer_lon, commodity, quantity_tonnes, available_markets):
    """
    This is the brain of the system - it recommends the best markets.
    """
    recommendations = []

    # Load the trained model and features list
    model = joblib.load(f'models/{commodity}_price_model.pkl')
    feature_columns = joblib.load(f'models/{commodity}_features.pkl')

    # Get today's date to predict for tomorrow
    prediction_date = datetime.now().date() + timedelta(days=1)

    for market in available_markets:
        # 1. Calculate distance to this market
        distance = calculate_distance(farmer_lat, farmer_lon, market['lat'], market['lon'])

        # 2. *** This is the key prediction step ***
        # To predict tomorrow's price, we need the features for tomorrow.
        # We use the most recent data we have from our dataset to create these features.
        latest_market_data = df_features[
            (df_features['market_name'] == market['name']) &
            (df_features['commodity'] == commodity)
        ].tail(1)

        if latest_market_data.empty:
            continue # Skip if we have no data for this market/commodity

        # Create the feature set for prediction
        features_for_prediction = pd.DataFrame(columns=feature_columns)
        features_for_prediction.loc[0, 'price_yesterday'] = latest_market_data['modal_price'].values[0]
        features_for_prediction.loc[0, 'price_last_week'] = latest_market_data['price_last_week'].values[0] # Approximation
        features_for_prediction.loc[0, 'price_avg_7days'] = latest_market_data['price_avg_7days'].values[0] # Approximation
        features_for_prediction.loc[0, 'day_of_week'] = prediction_date.weekday()
        features_for_prediction.loc[0, 'month'] = prediction_date.month
        features_for_prediction.loc[0, 'is_weekend'] = 1 if prediction_date.weekday() in [5, 6] else 0
        features_for_prediction.loc[0, 'arrivals_yesterday'] = latest_market_data['arrivals_qty'].values[0]
        features_for_prediction.loc[0, 'market_lat'] = market['lat']
        features_for_prediction.loc[0, 'market_lon'] = market['lon']

        # Make the price prediction
        predicted_price_per_quintal = model.predict(features_for_prediction)[0]
        predicted_price_per_kg = predicted_price_per_quintal / 100 # Convert from Quintal (100kg) to kg

        # 3. Calculate the net profit
        profit_info = calculate_net_profit(predicted_price_per_kg, quantity_tonnes, distance)

        recommendations.append({
            'market_name': market['name'],
            'distance_km': round(distance, 1),
            'predicted_price_kg': round(predicted_price_per_kg, 2),
            'expected_revenue': profit_info['revenue'],
            'total_costs': profit_info['total_costs'],
            'net_profit': profit_info['net_profit']
        })

    # Sort by net profit (highest first)
    recommendations.sort(key=lambda x: x['net_profit'], reverse=True)

    return recommendations[:3]  # Return top 3 markets

# --- Example Usage ---
# This is a sample list of markets our system knows about.
sample_markets = [
    {'name': 'Delhi_Mandi', 'lat': 28.6, 'lon': 77.2},
    {'name': 'Mumbai_Mandi', 'lat': 19.0, 'lon': 72.8},
    {'name': 'Pune_Mandi', 'lat': 18.5, 'lon': 73.8},
]

# Test the recommender with a farmer's details
recommendations = recommend_markets(
    farmer_lat=28.0,      # Farmer's location (near Delhi)
    farmer_lon=77.0,
    commodity='Tomato',
    quantity_tonnes=2,
    available_markets=sample_markets
)

print("--- Top Market Recommendations ---")
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. Market: {rec['market_name']}")
    print(f"   - Distance: {rec['distance_km']} km")
    print(f"   - Predicted Price: ₹{rec['predicted_price_kg']}/kg")
    print(f"   - Estimated Net Profit: ₹{rec['net_profit']:.0f}")
    print("-" * 20)

--- Top Market Recommendations ---
1. Market: Delhi_Mandi
   - Distance: 69.5 km
   - Predicted Price: ₹20.77/kg
   - Estimated Net Profit: ₹39747
--------------------
2. Market: Pune_Mandi
   - Distance: 1105.6 km
   - Predicted Price: ₹20.12/kg
   - Estimated Net Profit: ₹34318
--------------------
3. Market: Mumbai_Mandi
   - Distance: 1088.3 km
   - Predicted Price: ₹19.33/kg
   - Estimated Net Profit: ₹32820
--------------------
